## 기본 세팅

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath = '/content/drive/MyDrive/Project II/pet_data/'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image
import json
import warnings
from tqdm import tqdm
# %matplotlib inline 

In [ ]:
!pip install imagehash > /dev/null

In [ ]:
import glob
import sys
import cv2
import imageio
import joblib
import math
import warnings
import os
import torch
import imagehash
import matplotlib.patches as patches

In [ ]:
# Ignore Warnings
warnings.filterwarnings("ignore")

## 데이터 read

In [ ]:
df = pd.read_csv(filepath + 'train.csv')
df.head()

,Id,Subject Focus,Eyes,Face,Near,...,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,...,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,...,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,...,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,...,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,...,0,0,0,0,72


In [ ]:
# 추후 함수에서 이 값을 변환 시켜줄 예정
df['dog'] = 0 
df['cat'] = 0

In [ ]:
train_file_path = filepath + 'train/' # train 데이터 파일 주소

## YOLOv5를 사용해서 분류하기

 - 아래 내용은 [캐글 참고](https://www.kaggle.com/markwijkhuizen/petfinder-eda-yolov5-obj-detection-tfrecords)
 에서 가져왔음

In [ ]:
# Download YOLOV5 GitHub Repo
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 9876, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 9876 (delta 3), reused 5 (delta 3), pack-reused 9868
Receiving objects: 100% (9876/9876), 10.05 MiB | 22.27 MiB/s, done.
Resolving deltas: 100% (6872/6872), done.


In [ ]:
# Load Best Performing YOLOV5X Model
yolov5x6_model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-10-21 torch 1.9.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)



  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



  0%|          | 0.00/269M [00:00<?, ?B/s]

Fusing layers... 
Model Summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


- 이미지 분석 함수 YOLOv5를 사용하고 있음 자세한 사항은,,, 잘 모르겠습니다

In [ ]:
# Get Image Info
def get_image_info(file_path, plot=False):
    # Read Image
    image = imageio.imread(file_path)
    h, w, c = image.shape
    
    # 그래프는 현재 사용하지 않음. 기본값이 False
    if plot: # Debug Plots
        fig, ax = plt.subplots(1, 2, figsize=(8,8))
        ax[0].set_title('Pets detected in Image', size=16)
        ax[0].imshow(image)
        
    # Get YOLOV5 results using Test Time Augmentation for better result
    results = yolov5x6_model(image, augment=True)
    
    # Mask for pixels containing pets, initially all set to zero
    pet_pixels = np.zeros(shape=[h, w], dtype=np.uint8)
    
    # Dictionary to Save Image Info
    h, w, _ = image.shape # 높이, 너비, 차원
    image_info = { 
        'n_pets': 0, # Number of pets in the image
        'labels': [], # Label assigned to found objects # 현재 labels 에서 cat, dog만 사용하고 있음
        # 'thresholds': [], # confidence score
        # 'coords': [], # coordinates of bounding boxes
        'x_min': 0, # minimum x coordinate of pet bounding box
        'x_max': w - 1, # maximum x coordinate of pet bounding box
        'y_min': 0, # minimum y coordinate of pet bounding box
        'y_max': h - 1, # maximum x coordinate of pet bounding box
    }
    
    # Save found pets to draw bounding boxes
    pets_found = []
    
    # Save info for each pet
    for x1, y1, x2, y2, treshold, label in results.xyxy[0].cpu().detach().numpy():
        label = results.names[int(label)]
        if label in ['cat', 'dog']:
            image_info['n_pets'] += 1
            image_info['labels'].append(label)
            # image_info['thresholds'].append(treshold)
            # image_info['coords'].append(tuple([x1, y1, x2, y2]))
            image_info['x_min'] = max(x1, image_info['x_min'])
            image_info['x_max'] = min(x2, image_info['x_max'])
            image_info['y_min'] = max(y1, image_info['y_min'])
            image_info['y_max'] = min(y2, image_info['y_max'])
            
            # Set pixels containing pets to 1
            pet_pixels[int(y1):int(y2), int(x1):int(x2)] = 1
            
            # Add found pet
            pets_found.append([x1, x2, y1, y2, label])

    # 그래프는 현재 사용하지 않음. 기본값이 False 
    if plot:
        for x1, x2, y1, y2, label in pets_found:
            c = 'red' if label == 'dog' else 'blue'
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor=c, facecolor='none')
            # Add the patch to the Axes
            ax[0].add_patch(rect)
            ax[0].text(max(25, (x2+x1)/2), max(25, y1-h*0.02), label, c=c, ha='center', size=14)
                
    # Add Pet Ratio in Image
    image_info['pet_ratio'] = pet_pixels.sum() / (h*w)

    if plot:
        # Show pet pixels
        ax[1].set_title('Pixels Containing Pets', size=16)
        ax[1].imshow(pet_pixels)
        plt.show()
        
    return image_info

### 작성한 함수 : dog_cat_image

In [ ]:
# 고양이 강아지 분류 함수
# sdf : 데이터 프레임
# start : 데이터 프레임의 인덱스 시작
# end : 데이터 프레임의 인덱스 끝
# cat : 고양이 수 count
# dog : 강아지 수 count

def dog_cat_image(sdf, start, end, cat, dog):

  for idx, file_path in enumerate(tqdm(sdf['Id'][start:end])):
    imageFilePath = train_file_path + file_path + '.jpg' # '/content/drive/MyDrive/Project II/pet_data/' + 'train/' + file_path + '.jpg'
    new_name = file_path # 새로 지정할 이름 기본값은 원래 이미지 이름

    # get_image_info 함수를 통해 이미지의 정보를 가져옴
    info = get_image_info(imageFilePath)

    # 라벨링중 오류가 발생하는 경우가 있었음 (아무것도 검출되지 않는 경우, etc..)
    try:
      label = info['labels'][0] # 한 사진에 다른 종류의 동물은 없었으므로 한개만 봐도 충분하다고 생각
      if label == 'dog':
        new_name = label + str(dog) 
        shutil.copyfile(imageFilePath, train_file_path + 'dog/' + new_name + '.jpg') # shutill 함수를 이용해 복사하고, 새 이름으로 저장
        sdf['dog'][start + idx] = 1 # sdf['dog'] 값을 1로 지정
        dog += 1 # dog count 증가
      elif label == 'cat': # 강아지분류 와 같음
        new_name = label + str(cat)
        shutil.copyfile(imageFilePath, train_file_path + 'cat/' + new_name + '.jpg')
        sdf['cat'][start + idx] = 1
        cat += 1
    except:
      print('not exsist.')

    sdf['Id'][start + idx] = new_name # sdf['Id'] 값을 새 이름으로 바꾸어 저장

    print(f'cat : {cat}, dog : {dog}, idx : {start + idx}') # 잘 되고 있는지, 오류 발생시 확인하기 위해 print문 작성
  
  # 사실 전체 데이터프레임을 옮겼기 때문에 따로 리턴하는 의미가 크지는 않았으나
  # 혹시모를 오류를 대비하고자 전체를 return하고 전체를 input하였음.
  return sdf, cat, dog 

### 중간중간 함수를 조금 수정하였고, 마지막 함수 입니다.

In [ ]:
# cat, dog count 값 초기화
cat = 0
dog = 0

In [ ]:
tdf = df[['Id', 'dog', 'cat']] # 함수 계산시 메모리 용량을 줄이기 위해 필요한 컬럼만 가져왔음

In [ ]:
# 기본은 range(0, len(tdf), 1000)
for i in tqdm(range(9000, len(tdf), 1000)): 
  print('---')
  print(f'{i} 루프 시작')
  
  if i == 9000: # 1000단위로 끊기지 않기때문에 마지막 부분의 end값만 수정
    new_df, cat, dog = dog_cat_image(tdf, i, len(tdf), cat, dog)
  else: 
    new_df, cat, dog = dog_cat_image(tdf, i, i+1000, cat, dog) # 1000개씩 끊어서 작업
  new_df.to_csv(filepath + 'cat_dog.csv', index=False) # 중간 저장 필요..
  print(f'저장완료.')
  print('---')

  0%|          | 0/1 [00:00<?, ?it/s]

---
9000 루프 시작



  0%|          | 1/912 [00:00<10:27,  1.45it/s]

cat : 4832, dog : 4139, idx : 9000



  0%|          | 2/912 [00:01<09:48,  1.55it/s]

...

cat : 5323, dog : 4555, idx : 9909



100%|█████████▉| 911/912 [08:55<00:00,  1.68it/s]

cat : 5323, dog : 4556, idx : 9910



100%|██████████| 1/1 [08:55<00:00, 535.91s/it]

cat : 5323, dog : 4557, idx : 9911
저장완료.
---


## 저장하고 불러와서 합치기

In [ ]:
new_df.to_csv(filepath + 'cat_dog.csv', index=False) # 위의 for loop 내에서 저장을 이미 하였음.

In [ ]:
ndf = pd.read_csv(filepath + 'cat_dog.csv') # 데이터 로드

In [ ]:
ndf.head()

,Id,dog,cat
0,dog0,1,0
1,cat0,0,1
2,dog1,1,0
3,dog2,1,0
4,cat1,0,1


In [ ]:
# df는 원 train.csv 이고, 테스팅을 위해 temp값으로 복사 후 실행하였음
temp = df
temp['Id'] = ndf['Id']
temp['cat'] = ndf['cat']
temp['dog'] = ndf['dog']

In [ ]:
temp

,Id,Subject Focus,Eyes,Face,Near,...,Info,Blur,Pawpularity,cat,dog
0,dog0,0,1,1,1,...,0,0,63,0,1
1,cat0,0,1,1,0,...,0,0,42,1,0
2,dog1,0,1,1,1,...,0,0,28,0,1
3,dog2,0,1,1,1,...,0,0,15,0,1
4,cat1,0,0,0,1,...,0,0,72,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9907,cat5321,0,0,0,1,...,0,1,15,1,0
9908,dog4554,0,1,1,1,...,0,0,70,0,1
9909,cat5322,0,1,1,1,...,0,0,20,1,0
9910,dog4555,0,1,1,1,...,0,0,20,0,1


## 고양이, 강아지 별로 분류

In [ ]:
cat_df = temp[temp['cat'] == 1]
cat_df.tail()

,Id,Subject Focus,Eyes,Face,Near,...,Info,Blur,Pawpularity,cat,dog
9899,cat5318,0,1,1,1,...,0,0,31,1,0
9902,cat5319,0,1,1,1,...,0,0,50,1,0
9905,cat5320,0,1,1,1,...,0,0,31,1,0
9907,cat5321,0,0,0,1,...,0,1,15,1,0
9909,cat5322,0,1,1,1,...,0,0,20,1,0


In [ ]:
dog_df = temp[temp['dog'] == 1]
dog_df.tail()

,Id,Subject Focus,Eyes,Face,Near,...,Info,Blur,Pawpularity,cat,dog
9904,dog4552,0,1,1,1,...,0,0,44,0,1
9906,dog4553,0,1,1,1,...,0,0,31,0,1
9908,dog4554,0,1,1,1,...,0,0,70,0,1
9910,dog4555,0,1,1,1,...,0,0,20,0,1
9911,dog4556,0,1,1,1,...,0,0,30,0,1


In [ ]:
# 마지막 cat, dog 컬럼 삭제
dog_df = dog_df.drop(['cat', 'dog'], axis=1)
cat_df = cat_df.drop(['cat', 'dog'], axis=1)

In [ ]:
# 완성된 csv 파일 저장
cat_df.to_csv(filepath + 'cat.csv', index=False)
dog_df.to_csv(filepath + 'dog.csv', index=False)